# Pull amygdala-seeded gPPI estimates with all regions in the Harvard-Oxford Atlas


In [39]:
import nilearn
from nilearn import image
from nilearn import plotting
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import glob
import re

### Determine here which roi masks to pull betas for, then flatten them

This should be all masks in the Harvard-Oxford cortical and subcortical atlases

In [ ]:
maskPaths = glob.glob('../Parcellations/harvardOx/*.nii.gz')

masks = ['none'] * len(maskPaths)
for counter, maskPath in enumerate(maskPaths):
    masks[counter] = image.load_img(maskPath)
    masks[counter] = masks[counter].get_fdata()
    masks[counter] = masks[counter].flatten()

## Define a function to get gPPI estimates for any given ROI, run, ppi directory, contrast #, and image type (beta vs. tstat)

In [42]:
def getPpiRoi(subject, roiMask, runType, ppiDir, copeNum, imageType):

    # Pull for the emotional face and neutral face cope for each run
    statImg = image.load_img('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/%s%s.nii.gz'%(subject, runType, ppiDir, imageType, copeNum))
    statImg = statImg.get_fdata()
    statImg = statImg.flatten()
    
    # Mask the cope images and filter based on mask == 1, then take means      
    maskedStatImg= statImg[roiMask == 1]
    meanStatImg = np.mean(maskedStatImg)
    
    # return value
    return(meanStatImg)

# Pull HO amyg connectivity with all regions, with deconvolution

In [50]:
# data frame of all runs in feat
subFrame = pd.read_csv('../QA/allEmotionsFeatComplete.csv')
ppiDir = 'gppi_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg.feat'

subFrame = subFrame[subFrame['runType'] == 'fear'] 
for index, row in subFrame.iterrows():
    for counter, roiMask in enumerate(masks):
        maskPath = maskPaths[counter]
        #print(maskPath)
        maskPathSplit = maskPath.split('/')
        roiName = maskPathSplit[3][:-7]
        #print(roiName)
        if(os.path.isfile('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/cope1.nii.gz'%(row['name'], row['runType'], ppiDir))):
            subFrame.loc[index, 'phys_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'tstat')
            subFrame.loc[index, 'emotPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'tstat')
            subFrame.loc[index, 'neutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'tstat')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'tstat')

# to csv
subFrame.to_csv('gppi_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg_ALL_HO_REGIONS.csv', index = False)

# Pull HO amyg connectivity with all regions, without deconvolution

In [51]:
# data frame of all runs in feat
subFrame = pd.read_csv('../QA/allEmotionsFeatComplete.csv')
ppiDir = 'gppi_no_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg.feat'

subFrame = subFrame[subFrame['runType'] == 'fear'] 
for index, row in subFrame.iterrows():
    for counter, roiMask in enumerate(masks):
        maskPath = maskPaths[counter]
        #print(maskPath)
        maskPathSplit = maskPath.split('/')
        roiName = maskPathSplit[3][:-7]
        #print(roiName)
        if(os.path.isfile('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/cope1.nii.gz'%(row['name'], row['runType'], ppiDir))):
            subFrame.loc[index, 'phys_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'tstat')
            subFrame.loc[index, 'emotPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'tstat')
            subFrame.loc[index, 'neutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'tstat')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'tstat')

# to csv
subFrame.to_csv('gppi_no_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg_ALL_HO_REGIONS.csv', index = False)